In [1]:
import networkx as nx
import numpy as np
import scipy.sparse
import scipy.sparse.linalg
from scipy.sparse import coo_matrix,csr_matrix
import matplotlib.pyplot as plt

import timeit
%alias_magic t timeit

import sys
sys.path.insert(1, '../')

import SpringRank_tools as sr
import tools as tl
import cProfile

import pstats
from pstats import SortKey

%matplotlib inline
%load_ext autoreload
%aimport SpringRank_tools
%aimport tools
%autoreload 1

Created `%t` as an alias for `%timeit`.
Created `%%t` as an alias for `%%timeit`.


In [2]:
def get_network(N = 50,p = 0.1):
    adjacency_list = []
    for i in range(N):
        for j in range(N):
            if np.random.rand() < p:
                adjacency_list.append([i,j])
    A = np.zeros([N,N])
    for edge in adjacency_list:
        A[edge[0],edge[1]] += 1
    return A

In [4]:
A = get_network(N=15000,p=0.005)

In [5]:
R = csr_matrix(A)

In [6]:
%t R.sum(axis=0)

6.01 ms ± 64.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%t R.sum(axis=1)

3.23 ms ± 77.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
from scipy.sparse import coo_matrix,spdiags,csr_matrix
from scipy.sparse.csgraph import laplacian
def version_1(A):
    N = A.shape[0]
    k_in = A.sum(axis=0)
    k_out = A.sum(axis=1).transpose()
    # form the graph laplacian
    operator = csr_matrix(
        spdiags(k_out+k_in,0,N,N)-A-A.transpose()
        )
    return operator,k_in,k_out
def version_2(A):
    N = A.shape[0]
    k_in = A.transpose().sum(axis=1)
    k_out = A.sum(axis=1)
    # form the graph laplacian
    operator = csr_matrix(
        spdiags((k_out+k_in).transpose(),0,N,N)-A-A.transpose()
        )
    return operator
def version_3(A):
    N = A.shape[0]
    T = A.transpose()
    k_in = T.sum(axis=1)
    k_out = A.sum(axis=1)
    # form the graph laplacian
    operator = csr_matrix(
        spdiags((k_out+k_in).transpose(),0,N,N)-A-T
        )
    return operator
def version_4(A):
    N = A.shape[0]
    T = A.transpose()
    k_in = T.sum(axis=1)
    k_out = A.sum(axis=1)
    # form the graph laplacian
    operator = laplacian(A+T)
    return operator

In [20]:
Op = []
Op.append(version_1(R)[0])
Op.append(version_2(R))
Op.append(version_3(R))
Op.append(version_4(R))

In [21]:
table = np.zeros([4,4])
for idx,x in enumerate(Op):
    for idy,y in enumerate(Op):
        table[idx,idy] = (x!=y).nnz==0

In [22]:
table

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [23]:
%t version_1(R)

223 ms ± 6.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%t version_2(R)

215 ms ± 2.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%t version_3(R)

217 ms ± 7.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%t version_4(R)

546 ms ± 61.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
op,k_in,k_out = version_1(R)

In [41]:
def pad_1(k_in,k_out):
    solution_vector = np.append((k_out-k_in),np.matrix(0),axis=1).transpose()
    return solution_vector
def pad_2(k_in,k_out):
    solution_vector = np.zeros(len(k_out)+1)
    solution_vector[:len(k_out)] = (k_out-k_in).ravel().transpose()
    return solution_vector

### SpringRank

In [2]:
inadjacency = '../../data/US_CS_adjacency.dat'
G = tl.build_graph_from_adjacency(inadjacency)
nodes = list(G.nodes())
A= np.array(nx.to_numpy_matrix(G, nodelist=nodes))

In [67]:
N = 1000
beta = 0.1
alpha = 0.1
K = 20
A_n = sr.SpringRank_planted_network(N, beta, alpha, K, np.random)
A = np.array(nx.to_numpy_matrix(A_n))

In [71]:
%t sr.SpringRank(A, alpha=0)

172 ms ± 24.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [72]:
%t sr.SpringRank(A, alpha=0.1)

212 ms ± 19.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Sparse vs. Dense SpringRank

In [11]:
A = get_network(N=15000, p=0.005)
A_csr = csr_matrix(A)

In [12]:
%t sr.SpringRank(A)

30.4 s ± 1.72 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%t sr.SpringRank(A_csr)

1.21 s ± 31 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Online Ranking

In [26]:
def get_neighbors_1(G, P):
    N = set()
    m = G.shape[1]
    for x in P:
        for y in range(m):
            if y not in P and (G[x, y] != 0 or G[y, x] != 0):
                N.add(y)
    return list(N)

def get_neighbors_2(G, P):
    idx_0 = set(np.nonzero(G[P])[1])
    idx_1 = set(np.nonzero(G[:,P])[0])
    N = idx_0.union(idx_1) - set(P)
    return list(N)

In [31]:
A = get_network(N=1500,p=0.005)
P = np.random.randint(0, 1500, size=(30))

In [32]:
%t get_neighbors_1(A, P)

731 ms ± 55.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%t get_neighbors_2(A, P)

4.5 ms ± 320 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
